In [91]:
import pandas as pd
from datetime import date
from datetime import date, datetime, timedelta
import matplotlib.pyplot as plt

In [92]:
days_together = {
    "California": ["03-06-2022", "01-08-2022"],
    "Spain": ["09-08-2022", "19-08-2022"],
    "Paris": ["21-01-2023", "22-02-2023"],
    "Hawaii": ["03-11-2023", "10-12-2023"],
    "Italy": ["03-03-2024", "17-03-2024"],
    "Moving": ["09-09-2024", date.today().strftime("%d-%m-%Y")]
                 }

df = pd.DataFrame(days_together)

In [93]:
df_melted = df.melt(var_name="name", value_name="date")
df_melted["date"] = pd.to_datetime(df_melted["date"], format="%d-%m-%Y")
df_melted = df_melted.sort_values(by="date").reset_index(drop=True)
df_melted["ID"] = df_melted.index // 2 + 1
df_melted["date_type"] = df_melted.groupby("ID").cumcount() + 1
df_pivoted = df_melted.pivot(index="ID", columns="date_type", values="date").reset_index()
df_pivoted.columns = ["ID", "date1", "date2"]
df_final = df_pivoted.merge(df_melted[["ID", "name"]].drop_duplicates(), on="ID")
df_final["number of days"] = (df_final["date2"] - df_final["date1"]).dt.days
df_final["together"] = df_final["number of days"] > 0

def fill_dates(row, next_row_date1=None):
    filled_rows = []
    if next_row_date1:
        filled_rows.append({
            "date1": row["date2"] + timedelta(days=1),
            "date2": next_row_date1 - timedelta(days=1),
            "name": None,
            "number of days": 0,
            "together": False
        })
    return filled_rows

filled_data = []
for i, row in df_final.iterrows():
    next_row_date1 = df_final.loc[i + 1, "date1"] if i + 1 < len(df_final) else None
    filled_data.extend(fill_dates(row, next_row_date1))

filled_df = pd.DataFrame(filled_data)
combined_df = pd.concat([df_final, filled_df], ignore_index=True).sort_values(by=["date1"]).reset_index(drop=True)
combined_df["number of days"] = (combined_df["date2"] - combined_df["date1"]).dt.days
combined_df = combined_df.drop(columns=["ID"])
combined_df

,date1,date2,name,number of days,together
0,2022-06-03,2022-08-01,California,59,True
1,2022-08-02,2022-08-08,None,6,False
2,2022-08-09,2022-08-19,Spain,10,True
3,2022-08-20,2023-01-20,None,153,False
4,2023-01-21,2023-02-22,Paris,32,True
5,2023-02-23,2023-11-02,None,252,False
6,2023-11-03,2023-12-10,Hawaii,37,True
7,2023-12-11,2024-03-02,None,82,False
8,2024-03-03,2024-03-17,Italy,14,True
9,2024-03-18,2024-09-08,None,174,False


In [94]:
combined_df.groupby("together")["number of days"].sum()


stats_out = pd.DataFrame({
    "Days Together": combined_df.groupby("together")["number of days"].sum().iloc[1],
    "Days Separated": combined_df.groupby("together")["number of days"].sum().iloc[0],
    "Days Till 1": combined_df.groupby("together")["number of days"].sum().iloc[0] - combined_df.groupby("together")["number of days"].sum().iloc[1] + 1,
    "Days Till One Year": 365 - combined_df.groupby("together")["number of days"].sum().iloc[1],
}, index=[0])

stats_out["One Year Date"] = datetime.today() + pd.Timedelta(days=int(stats_out["Days Till One Year"].values[0]))
stats_out["Plus One Date"] = datetime.today() + pd.Timedelta(days=int(stats_out["Days Till 1"].values[0]))
stats_out

,Days Together,Days Separated,Days Till 1,Days Till One Year,One Year Date,Plus One Date
0,256,667,412,109,2025-04-10 19:27:26.063078,2026-02-07 19:27:26.063704


In [95]:
combined_df["date1"] = pd.to_datetime(combined_df["date1"])
combined_df["date2"] = pd.to_datetime(combined_df["date2"])
combined_df["adjusted days"] = combined_df["number of days"] * combined_df["together"].apply(lambda x: 1 if x else -1)
combined_df["name"] = combined_df["name"].apply(lambda x: "Long Distance" if not x else x)
combined_df["moving average"] = combined_df["adjusted days"].cumsum()

combined_df

,date1,date2,name,number of days,together,adjusted days,moving average
0,2022-06-03,2022-08-01,California,59,True,59,59
1,2022-08-02,2022-08-08,Long Distance,6,False,-6,53
2,2022-08-09,2022-08-19,Spain,10,True,10,63
3,2022-08-20,2023-01-20,Long Distance,153,False,-153,-90
4,2023-01-21,2023-02-22,Paris,32,True,32,-58
5,2023-02-23,2023-11-02,Long Distance,252,False,-252,-310
6,2023-11-03,2023-12-10,Hawaii,37,True,37,-273
7,2023-12-11,2024-03-02,Long Distance,82,False,-82,-355
8,2024-03-03,2024-03-17,Italy,14,True,14,-341
9,2024-03-18,2024-09-08,Long Distance,174,False,-174,-515


In [102]:
import plotly.express as px
# Plot the moving average over time with context labels
fig = px.line(
    combined_df,
    x="date2",
    y="moving average",
    title="Days Together vs. Apart",
    labels={"moving average": "Days Together vs. Apart", "date2": "Date"},
    hover_data={"name": True, "adjusted days": True}
)
fig.show()